In [2]:
pip install pyautogen
pip install xgboost

  Using cached pyautogen-0.2.28-py3-none-any.whl.metadata (25 kB)
Using cached pyautogen-0.2.28-py3-none-any.whl (284 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent

* Downloaded **Mistral-7B-Instruct-v0.1-GGUF** LLM via LM Studio from Hugging Face.

* Deployed LLM model on local server. 

## Using ConversableAgent

In [6]:
mistral={
"config_list":[{
    "model":"TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf",
    "base_url":"http://localhost:1234/v1",
    "api_key":"lm-studio"
},],
"cache_seed":None
}


Bill= ConversableAgent(
    name="assistant",
    llm_config=mistral,
    system_message="You are Bill and you are a comedian."
)


Burr= ConversableAgent(
    name="assistant",
    llm_config=mistral,
    system_message="You are Burr and you are a part of two person comedy show."
)


chat_result = Bill.initiate_chat(Burr,message="Burr, tell me a joke",max_turns=2)


assistant (to assistant):

Burr, tell me a joke

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to assistant):

 Sure thing! Here's one for you:

Why did the tomato turn red?

Because it saw the salad dressing!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to assistant):

 That's a classic, Timmy. Do you have any other good ones up your sleeve?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to assistant):

 Of course I do! How about this one:

Why was the math book sad?

Because it had too many problems!

--------------------------------------------------------------------------------


## AssistantAgent and UserProxyAgent

In [3]:
assistant = AssistantAgent("assistant", llm_config={"config_list": mistral['config_list']})
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended
user_proxy.initiate_chat(assistant, message="Tell me a Joke.")

user_proxy (to assistant):

Tell me a Joke.

--------------------------------------------------------------------------------
assistant (to user_proxy):

 Sure! Here's a joke for you:

Why was the math book sad? Because it had too many problems. 

I hope you found that joke amusing! Let me know if you need anything else.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Tell me a Joke.', 'role': 'assistant'}, {'content': " Sure! Here's a joke for you:\n\nWhy was the math book sad? Because it had too many problems. \n\nI hope you found that joke amusing! Let me know if you need anything else.", 'role': 'user'}], summary=" Sure! Here's a joke for you:\n\nWhy was the math book sad? Because it had too many problems. \n\nI hope you found that joke amusing! Let me know if you need anything else.", cost={'usage_including_cached_inference': {'total_cost': 0, 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf': {'cost': 0, 'prompt_tokens': 498, 'completion_tokens': 47, 'total_tokens': 545}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf': {'cost': 0, 'prompt_tokens': 498, 'completion_tokens': 47, 'total_tokens': 545}}}, human_input=['exit'])

## Groupchat (Planner, Admin, Analyst, Engineer)


Engineer: write code

Analyst: Analyses and gets insights

Planner: Suggests plan

Executor: Execute the code

In [3]:
mistral={
"config_list":[{
    "model":"TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf",
    "base_url":"http://localhost:1234/v1",
    "api_key":"lm-studio"
},],
"cache_seed":None
}

In [4]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=mistral,
    system_message="""Engineer. You follow an approved plan. You write python code to solve tasks. The user can't modify your code. Do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
""",
)

scientist = autogen.AssistantAgent(
    name="Analyst",
    llm_config=mistral,
    system_message="""Analyst. You follow an approved plan. You are able to draw conclusion from codes and graphs. You don't write code.""",
)
#########################################
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin, until admin approval.
The plan may involve an engineer who can write code and an Analyst who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a Analyst.
""",
    llm_config=mistral,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=30
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=mistral)

In [7]:
user_proxy.initiate_chat(
    planner,
    message="""
Should a businessman invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks using fbprophet library. If stock close prices are increasing for next 5 days, invest in Apple stocks, otherwise don't.
""",
)

Admin (to Planner):


Should a businessman invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks using fbprophet library. If stock close prices are increasing for next 5 days, invest in Apple stocks, otherwise don't.


--------------------------------------------------------------------------------
Planner (to Admin):

 Plan:

1. Collect historical data for Apple stock prices over a specified period of time (e.g., 1 year).
2. Clean and preprocess the data using pandas library.
3. Train a time-series model using fbprophet library on the preprocessed data to predict future stock prices.
4. Use the predictions to determine if the stock prices are increasing for the next 5 days.
5. If the stock prices are predicted to increase, recommend investing in Apple stocks. Otherwise, do not recommend investing.
6. Review the plan and make any necessary revisions based on feedback from admin.

Roles:

1. Engineer - responsible for writing code for data collection, pr

In [ ]:
########################################################################################################################################
##                                                      System heated                                                                 ##
########################################################################################################################################

## RAG Implementation in Autogen

In [2]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

task='qa'

In [27]:
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
    code_execution_config=False
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "qa",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
    },
    code_execution_config=False,
)

[autogen.oai.client: 06-14 22:47:12] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


c:\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
assistant.reset()
ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem="What is autogen?")

2024-06-15 01:23:24,770 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Use the existing collection `autogen-docs`.


Trying to create collection.


2024-06-15 01:23:25,919 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 1 chunks.
Number of requested results 20 is greater than number of elements in index 3, updating n_results = 3


VectorDB returns doc_ids:  [['2bea6e64', '36d6eefd', '600897af']]
Adding content of doc 2bea6e64 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is autogen?

Context is: # Getting Started | AutoGen

Getting Started | AutoGen

[Skip to main content](#__docusaurus_skipToContent_fallback)What's new in AutoGen? Read [this blog](/autogen/blog/2024/03/03/AutoGen-Update) for an overview of updates[![AutoGen](/autogen/img/ag.svg)![AutoGen](/autogen/img/ag.svg)**AutoGen**](/autogen/)[Docs](#)* [Getting Started](/autogen/docs/Getting-Started)
* [Installation](/autogen/docs/installation/)
* [Tutorial](/autogen/docs/tutorial/introduction)
* [User Guide](/autogen/docs/topics)
* [API Reference]

#### task = "code"

In [4]:
mistral={
"config_list":[{
    "model":"TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf",
    "base_url":"http://localhost:1234/v1",
    "api_key":"lm-studio"
},],
"cache_seed":None
}


In [8]:
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=mistral,
    code_execution_config=False
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "code",
        "docs_path":"https://github.com/aakash1104/Graph-Algorithms/README.md"
        "get_or_create":True
    },
    code_execution_config=False,
)

c:\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
assistant.reset()
ragproxyagent.initiate_chat(assistant, problem="Write a code for Dijkstra's algorithm and explain the code.")

ragproxyagent (to assistant):

Write a code for Dijkstra's algorithm and explain the code.

--------------------------------------------------------------------------------


In [ ]:
#######################################################################################
##                      System heated up and crashed for last part                   ##
#######################################################################################